# GET DATA FROM ALPHA VANTAGE

In [13]:
from datetime import datetime, timedelta
import os

import requests
import pandas as pd

In [19]:
KEY2 = os.getenv("HSTH_AV_KEY")


SOURCE="alphavantage"
API_KEY = KEY2
YYYY="2023"
MM="12"
INTERVAL="5min"
TICKER="SPY"

In [20]:
def get_historicals_from_alphavantage():
    print("getting data")

    # url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=SPY&interval=5min&apikey={API_KEY}"
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={TICKER}&interval={INTERVAL}&month={YYYY}-{MM}&outputsize=full&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data

In [21]:
data = get_historicals_from_alphavantage()

getting data


In [28]:
!ls ../data

2022-10-SPY_5min_alphavantage.csv  2023-3-SPY_5min_alphavantage.csv
2022-11-SPY_5min_alphavantage.csv  2023-4-SPY_5min_alphavantage.csv
2022-12-SPY_5min_alphavantage.csv  2023-5-SPY_5min_alphavantage.csv
2023-10-SPY_5min_alphavantage.csv  2023-6-SPY_5min_alphavantage.csv
2023-11-SPY_5min_alphavantage.csv  2023-7-SPY_5min_alphavantage.csv
2023-12-SPY_5min_alphavantage.csv  2023-8-SPY_5min_alphavantage.csv
2023-1-SPY_5min_alphavantage.csv   2023-9-SPY_5min_alphavantage.csv
2023-2-SPY_5min_alphavantage.csv


In [39]:
filename = "2022-8-SPY_5min_alphavantage.csv"
filename_n_path = f"../data/{filename}"
data = pd.read_csv(filename_n_path)
data

,Unnamed: 0,1. open,2. high,3. low,4. close,5. volume
0,2025-03-10 20:00:00,560.580,560.58,560.5800,560.5800,3586303
1,2025-03-10 19:55:00,558.600,558.71,558.3800,558.5118,21280
2,2025-03-10 19:50:00,558.770,558.85,558.5400,558.6500,17102
3,2025-03-10 19:45:00,558.275,558.88,558.2025,558.8175,25557
4,2025-03-10 19:40:00,557.960,558.41,557.7700,558.2750,31580
...,...,...,...,...,...,...
3855,2025-02-10 04:20:00,603.060,603.14,603.0000,603.0300,1712
3856,2025-02-10 04:15:00,602.990,603.15,602.9600,603.0200,2131
3857,2025-02-10 04:10:00,602.810,603.00,602.7600,603.0000,856
3858,2025-02-10 04:05:00,602.850,602.94,602.6900,602.8700,2352


In [40]:
df = pd.read_csv(filename_n_path)
print(df.head())

            Unnamed: 0  1. open  2. high    3. low  4. close  5. volume
0  2025-03-10 20:00:00  560.580   560.58  560.5800  560.5800    3586303
1  2025-03-10 19:55:00  558.600   558.71  558.3800  558.5118      21280
2  2025-03-10 19:50:00  558.770   558.85  558.5400  558.6500      17102
3  2025-03-10 19:45:00  558.275   558.88  558.2025  558.8175      25557
4  2025-03-10 19:40:00  557.960   558.41  557.7700  558.2750      31580


In [41]:
df["Unnamed: 0"][0][:10]

'2025-03-10'

In [46]:
from datetime import datetime, timedelta
(datetime.today()- timedelta(1)).strftime('%Y-%m-%d')

'2025-03-10'

In [ ]:
df = pd.DataFrame.from_dict(data["Time Series (5min)"], orient="index")
print(df.head())


KeyError: 'Time Series (5min)'

In [6]:
csv_name = f"{YYYY}-{MM}-{TICKER}_{INTERVAL}_{SOURCE}.csv"

In [7]:

df.to_csv(f"../data/{csv_name}")

# UPLOAD DATA TO GOOGLE CLOUD BUCKET

In [8]:
from google.cloud import storage


In [9]:


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket:        bucket_name = "your-bucket-name"
    # The path to your file to upload:  source_file_name = "local/path/to/file"
    # The ID of your GCS object:        destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [10]:
bucket_name = "cgpdata"
source_file_name = f"../data/{csv_name}"
gcpbucket_folder_string = "hist"
destination_blob_name = f"{gcpbucket_folder_string}/{csv_name}"

In [12]:
upload_blob(bucket_name, source_file_name, destination_blob_name)

File ../data/2023-12-SPY_5min_alphavantage.csv uploaded to hist/2023-12-SPY_5min_alphavantage.csv.
